In [8]:
import pandas as pd
import seaborn as sns

data = pd.read_csv("df.csv")
data_corr = data.drop(columns=["Company", "Sector"])

data_corr.corr()



,CR,DA,FL,OPM,RT,TAT,DE,Y,PT
CR,1.000000,-0.225917,-0.179232,0.284529,-0.151203,-0.123665,-0.141136,-0.018690,-0.040483
DA,-0.225917,1.000000,0.340320,0.015032,0.206620,-0.083691,0.464786,0.164599,-0.120993
FL,-0.179232,0.340320,1.000000,-0.112430,-0.076335,0.156169,0.968612,0.759323,0.032588
OPM,0.284529,0.015032,-0.112430,1.000000,0.051672,-0.298701,-0.050014,0.051308,0.021409
RT,-0.151203,0.206620,-0.076335,0.051672,1.000000,0.206580,-0.065267,-0.046290,0.097708
TAT,-0.123665,-0.083691,0.156169,-0.298701,0.206580,1.000000,0.094869,0.301222,0.017872
DE,-0.141136,0.464786,0.968612,-0.050014,-0.065267,0.094869,1.000000,0.724604,-0.025348
Y,-0.018690,0.164599,0.759323,0.051308,-0.046290,0.301222,0.724604,1.000000,-0.001363
PT,-0.040483,-0.120993,0.032588,0.021409,0.097708,0.017872,-0.025348,-0.001363,1.000000


# Část 1: Největší vliv na indikátor Y akcie má indikátor FL (Financial Leverage, finanční páka). Hodnota korelačního koeficientu je 0,759323. To znamená, že je zde předpoklad, že s rostoucí hodnotou indikátoru FL poroste také hodnota indikátoru Y - tedy poroste hodnota akcie dané společnosti. 

In [10]:
import statsmodels.formula.api as smf

formula = "Y ~ FL + OPM + TAT"
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
res.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     59.51
Date:                Wed, 15 Nov 2023   Prob (F-statistic):           7.99e-22
Time:                        21:27:14   Log-Likelihood:                -377.99
No. Observations:                 100   AIC:                             764.0
Df Residuals:                      96   BIC:                             774.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4188      2.865     -3.637      0.000     -16.106      -4.732
FL             2.5281      0.208     12.149      0.000       2.115       2.941
OPM           24.8478      7.542      3.295      0.001       9.878      39.818
TAT            8.5518      2.205      3.879      0.000       4.176      12.928
==============================================================================
Omnibus:                       26.748   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               76.291
Skew:                           0.876   Prob(JB):                     2.71e-17
Kurtosis:                       6.904   Cond. No.                         50.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Statisticky nevýznamné se ukázaly koeficienty u indikátorů CR, DA, RT, DE a PT (p-hodnota > 0,05). Tyto byly z regresního modelu vyloučeny.

In [31]:
data["Y_estimate"] = res.predict(data)
data["Y_difference"] = data["Y_estimate"] - data["Y"]

data.head()

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_estimate,Y_difference
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials,0.502359,-1.447641
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services,23.356512,16.096512
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services,6.944718,-0.835282
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive,15.708512,9.538512
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical,1.701358,1.701358


In [24]:
data["Y_difference"].sort_values(ascending=False)

79    34.809545
98    23.278968
6     22.850015
1     16.096512
56    15.376012
        ...    
54   -25.819240
81   -27.609819
39   -29.286830
75   -32.491447
45   -42.540694
Name: Y_difference, Length: 100, dtype: float64

In [32]:
data.at[79, "Company"]

'SYXLy'

# Největší rozdíl mezi odhadovanou hodnotou akcie (Y_estimate) a skutečnou hodnotou akcie (Y) je u akcie SYXLy. U této náš model predikuje výrazně vyšší tržní hodnotu (vyšší o 34,809545), než jakou má ve skutečnosti.